# Self-Attention Generative Adversarial Networks (2018.05.)


<br> 2018/09/01
<br> Jaehwi Park

## 1. Introduction

최근까지의 GAN모델은 texture patterns는 잘 잡아내는 반면, geometric or structural patterns을 잘 잡아내지 못한다는 문제점이 있다고 합니다.
이에 대한 가장 큰 이유로, Conv Layer가 Localized된 정보를 주로 전달하기 때문이라고 합니다. 이를 해결하기 위해서 Receptive Field 또는 Stride를 키울 수 있지만, 이에 따른 부작용도 함께 발생합니다. 그래서 __Self-Attention__ 방법을 사용합니다.

![Figure1](./Figure1.png)

본 논문에서는 Self-attention + GAN (SAGAN)을 제안합니다. Self-attention module 덕분에 long range & multi-level dependencies across
image region을 고려할 수 있게 됐습니다. 

## 2. Related Work

### 2.1 GAN

    - skip

### 2.2 Attention Models

    - 더 자세하게 만들 자신이 없어서 링크 겁니다...

https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/

https://jalammar.github.io/illustrated-transformer/

## 3. Self-Attention Generative Adversarial Networks

"GAN + Attention"이 이 논문의 Main Contribution 입니다.

![Figure2](Figure2.png)

> 1. 일단 Conv를 통과시킨 Feature Map을 Channel X (W*H) 차원으로 바꿔줍니다 $ x \in \mathbb{R}^{C X N} $
> 2. Attention Matrix를 각각 준비합니다. (QUERY~F(x), KEY~G(x), VALUE~H(x))
> 3. Attention output을 계싼합니다. 
    - $o_j = (F^T \otimes G) \odot H$ 
> 4. 쉬운 task로 먼저 학습시키고, 점진적으로 어려운 task를 학습시킬 수 있도록 합니다.
    - $y_i = \gamma o_i + x_i$
    - where $\gamma$ is innitilized as 0
> 5. Adversarial Loss는 Hinge Loss version을 사용합니다.

## 4. Techniques to stabilize GAN training
### 4.1 Spectral Normalization for both generator and discriminator

> 1. lipschitz를 반영하는 GAN모델들이 다음과 같음
    - GAN -> WGAN -> WGAN-GP -> Spectral Normalization(SN)
> 2. SN논문(2018.02)에서는 Discriminator에만 반영하는 것을 제안했는데, Generator에도 SN을 쓰면 좋을 것 같아서 썼다고 합니다.

### 4.2 Imbalanced learning rate for generator and discriminator updates

> SN논문에서 Regularized Discriminator를 계산할 때, 학습이 둔화되는 문제가 있었습니다. <br>
> 그래서 TTUR이라는 기법을 사용하여, [Discriminator Steps >= Generator Steps] 하도록 하여 Discriminator의 학습 속도를 향상시킵니다. <br>
> 같은 wall-clock time에서 더 좋은 결과를 만들어 낼 수 있었다고 합니다.

In [1]:
## [[ Implementation ~ https://github.com/heykeetae/Self-Attention-GAN/blob/master/sagan_models.py]]

class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim,activation):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        self.activation = activation
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out,attention

NameError: name 'nn' is not defined